In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # graphing stuff
from datetime import datetime # time
import math
import time
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator

# For linear regression model
import sklearn # python library for linear and other models
import warnings # to supress future warnings
from sklearn import linear_model

# LSTM for international airline passengers problem with regression framing
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
plt.style.use('seaborn')
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.

In [ ]:
# We are visualizing the impact of Netflix, Amazon Prime Video and other streaming services on the movie industry.

In [ ]:
print('Hi eddie!')

In [ ]:
netflix = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')

In [ ]:
netflix.sort_values('release_year', ascending=False, inplace=True)

# converting Date_added column to datetime object
netflix['date_added'] = pd.to_datetime(netflix['date_added'])
netflix.sort_values('date_added', ascending=True, inplace=True)
netflix.head(10)

In [ ]:
# Copy of dataframe
netflix_1 = netflix

# Making a list of all rows to delete
droppings = netflix_1[netflix_1['type'] != 'Movie'].index

# Delete these row indexes from dataFrame
netflix_1.drop(droppings, inplace=True)
netflix_movies = netflix_1
netflix_movies.sort_values('date_added', ascending=False, inplace=True)

In [ ]:
netflix_movies.drop(netflix_movies.index[4264])
netflix.sort_values('date_added', ascending=True, inplace=True)
netflix_movies.head()

***Movie Data***

In [ ]:
movie_data = pd.read_csv('/kaggle/input/moviedata/MovieData.csv')

In [ ]:
sum_column = movie_data["domestic_box_office"] + movie_data["international_box_office"]
movie_data["worldwide_box_office"] = sum_column
movie_data.head()

In [ ]:
movie_data = movie_data[["movie_name", "production_year", "movie_odid", "production_budget", "domestic_box_office", "international_box_office", "worldwide_box_office", "rating", "creative_type", "source", "production_method", "genre", "sequel", "running_time"]]
movie_data.head(10)

In [ ]:
movie_data.plot(kind='scatter', x='production_budget', y='international_box_office', color='blue')

In [ ]:
difference_column = movie_data["worldwide_box_office"] - movie_data["production_budget"]
movie_data["box_office_gross"] = difference_column

In [ ]:
movie_data = movie_data[["movie_name", "production_year", "movie_odid", "production_budget", "domestic_box_office", "international_box_office", "worldwide_box_office", "box_office_gross", "rating", "creative_type", "source", "production_method", "genre", "sequel", "running_time"]]
movie_data.tail(10)

In [ ]:
# netflix_movies.tail()
netflix_movies = netflix_movies.drop([6231], axis=0)
netflix_movies.tail()

In [ ]:
netflix_movies['year_added'] = pd.to_datetime(netflix_movies['date_added'], format='%Y')
netflix_movies = netflix_movies[["show_id", "type", "title", "director", "cast", "country", "date_added", "year_added", "release_year", "rating", "duration", "listed_in", "description"]]

In [ ]:
bruh = netflix_movies['year_added']
new = []
for i in bruh:
    j = int(str(i)[:4])
    new.append(j)
    
netflix_movies['year_added'] = new
netflix_movies.head()

*** > *****Netflix Movies Grouped and Graphed*****> **

In [ ]:
group_year = netflix_movies.groupby(['year_added']).count()
group_year_list = netflix_movies.groupby(['year_added'])['date_added'].count()
group_year_list = group_year_list.values.tolist()
group_year['total_movies'] = group_year_list
group_year = group_year.reset_index()
group_year

In [ ]:
# Graphing Netflix Movies By Year

group_year.plot(kind='bar', x='year_added', y='total_movies')
plt.title('Number of Movies Added to Netflix Each Year')
plt.xlabel('Year')
plt.ylabel('# of Movies Added')

> Actual Movies Grouped and Graphed******

In [ ]:
movie_data.head()

In [ ]:
movie_data.info()

In [ ]:
year_grouped = movie_data.groupby(['production_year']).count()
year_grouped_list = movie_data.groupby(['production_year'])['production_year'].count()
year_grouped_list = year_grouped_list.values.tolist()
year_grouped['total_movies'] = year_grouped_list
year_grouped = year_grouped.reset_index()
year_grouped

In [ ]:
# Graphing Movies By Year

year_grouped.plot(kind='bar', x='production_year', y='total_movies')
plt.title('Number of Movies Made Each Year')
plt.xlabel('Year')
plt.ylabel('# of Movies')

# Graphing Netflix Movies By Year

group_year.plot(kind='bar', x='year_added', y='total_movies')
plt.title('Number of Movies Added to Netflix Each Year')
plt.xlabel('Year')
plt.ylabel('# of Movies Added')

> Adding Another Dataset******

In [ ]:
years = pd.read_csv('/kaggle/input/moviesbyyear/GPG - TOP Box Office Grossers All Time - Apr 2020 - Movies by Year.csv')

In [ ]:
years.head(10)

In [ ]:
years.sort_values('Year', ascending=False, inplace=True)
# years = years.drop(years.index[:8])
years

In [ ]:
# years = years.drop('Unnamed: 0', axis=1)
# years = years.drop('Annual\nStats', axis=1)
years.sort_values('Year', ascending=True, inplace=True)
years.head()

In [ ]:
years.rename(columns = {'Year':'year', 'Total\nMovies':'total-movies', 'Average\nProduction\nBudget':'average-production-budget', 'Combined\nWorldwide\nBox Office':'combined-worldwide-box-office', 'No. 1 Movie':'best-movie'},
             inplace = True)
years.head()

In [ ]:
# years['total-movies'] = years['total-movies'].str.replace(',', '')
# years['total-movies'] = years['total-movies'].astype(int)
years.info()

In [ ]:
# years['combined-worldwide-box-office'] = years['combined-worldwide-box-office'].str.replace(',', '')
# years['combined-worldwide-box-office'] = years['combined-worldwide-box-office'].str.replace('$', '')
# years['combined-worldwide-box-office'] = years['combined-worldwide-box-office'].astype(int)
years['combined-worldwide-box-office']

In [ ]:
years['year'] = years['year'].astype(str).astype(int)
years.head()

In [ ]:
# Graphing Movies by Year

years.plot(kind='bar', x='year', y='total-movies', color='red')
plt.title('Number of Movies Made Every Year')
plt.xlabel('Year')
plt.ylabel('# of Movies Made')

# Graphing Worldwide Box Office Per Year

years.plot(kind='bar', x='year', y='combined-worldwide-box-office', color='green')
plt.title('Worldwide Box Office Gross Per Year ')
plt.xlabel('Year')
plt.ylabel('Worldwide Box Office')

# Average Gross Per Year

years['avg-gross-per-movie'] = years['combined-worldwide-box-office'] / years['total-movies']
years.plot(kind='bar', x='year', y='avg-gross-per-movie', color='purple')
plt.title('Average Gross of Movie Per Year')
plt.xlabel('Year')
plt.ylabel('Average Gross of Movie')

# Graphing Netflix Movies By Year

group_year.plot(kind='bar', x='year_added', y='total_movies')
plt.title('Number of Movies Added to Netflix Each Year')
plt.xlabel('Year')
plt.ylabel('# of Movies Added')

In [ ]:
netflix_fake = group_year['total_movies'].tolist()
netflix_0 = [0, 0, 0, 0, 0, 0, 0, 0]
netflix = netflix_0 + netflix_fake
netflix

In [ ]:
# Graphing Movies by Year

years.plot(kind='bar', x='year', y='total-movies', color='red')
plt.title('Number of Movies Made Every Year')
plt.xlabel('Year')
plt.ylabel('# of Movies Made')

# Graphing Number of Netflix Movies Added
group_year.plot(kind='bar', x='year_added', y='total_movies')
plt.title('Number of Movies Added to Netflix Each Year')
plt.xlabel('Year')
plt.ylabel('# of Movies Added')

In [ ]:
# Doing this

# set width of bar
barWidth = 0.35
 
# set height of bar
movies = years['total-movies'].tolist()
netflix_fake = group_year['total_movies'].tolist()
netflix_0 = [0, 0, 0, 0, 0, 0, 0, 0]
netflix = netflix_0 + netflix_fake
 
# Set position of bar on X axis
r1 = np.arange(len(movies))
r2 = [x+0.1 + barWidth for x in r1]
 
# Make the plot
plt.bar(r1+0.02, movies, color='blue', width=barWidth, edgecolor='white', label='movies')
plt.bar(r2, netflix, color='red', width=0.25, edgecolor='white', label='netflix-movies-added')
 
# Add xticks on the middle of the group bars
plt.xlabel('year', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(movies))], ['2000', '2002', '2004', '2006', '2008', '2010', '2012', '2014', 
                                                        '2016', '2018', '2020'])
plt.locator_params(axis='x', nbins=20)
 
# Create legend & Show graphic
plt.legend()
plt.show()

> Machine Learning - Linear Regression****

In [ ]:
# https://towardsdatascience.com/a-beginners-guide-to-linear-regression-in-python-with-scikit-learn-83a8f7ae2b4f
# https://medium.com/@gr33ndata/learn-regressions-analysis-23b789bf2c36

In [ ]:
# EXAMPLE OF HOW TO USE LINEAR REGRESSION TO PREDICT THE FUTURE

# We are gonna use Scikit's LinearRegression model
from sklearn.linear_model import LinearRegression

# Your input data, X and Y are lists (or Numpy Arrays)
x = [[2,4],[3,6],[4,5],[6,7],[3,3],[2,5],[5,2]]
y = [14,21,22,32,15,16,19]

# Initialize the model then train it on the data
genius = LinearRegression()
genius.fit(x,y)

# Predict the corresponding value of Y for X = [8,4]
genius.predict([[8,4]])

In [ ]:
group_year.info()

In [ ]:
without_2020 = group_year[:12]

years = [2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030]
predicted_movies_added = []
for i in years:
    x = without_2020[['year_added']].to_numpy()
    x.reshape(-1, 1)
    y = without_2020['total_movies']

    model = LinearRegression()
    model.fit(x,y)

    prediction = model.predict([[i]])
    new_form = int(prediction)
    predicted_movies_added.append(new_form)
    
print(predicted_movies_added)
group_year

In [ ]:
column = group_year['total_movies'].tolist()
old_column = column[:12]
new_column = old_column + predicted_movies_added
new_column

In [ ]:
years = [2008, 2009, 2010, 
         2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 
         2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030]

data = {'year_added' : years, 'total_movies' : new_column}
new_netflix = pd.DataFrame(data)
new_netflix

In [ ]:
new_netflix.plot(kind='bar', x='year_added', y='total_movies', color='red')

# ***SVM MODEL***

In [ ]:
# Import gang

x = without_2020[['year_added']].to_numpy()
x.reshape(-1, 1)
y = without_2020['total_movies']

from sklearn.model_selection import train_test_split
X_train_confirmed, X_test_confirmed, y_train_confirmed, y_test_confirmed = train_test_split(x, y, test_size=0.15, shuffle=False)

In [ ]:
# Building the SVM model

kernel = ['poly', 'sigmoid', 'rbf']
c = [0.01, 0.1, 1, 10]
gamma = [0.01, 0.1, 1]
epsilon = [0.01, 0.1, 1]
shrinking = [True, False]
svm_grid = {'kernel': kernel, 'C': c, 'gamma' : gamma, 'epsilon': epsilon, 'shrinking' : shrinking}

svm = SVR()
svm_search = RandomizedSearchCV(svm, svm_grid, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=1, verbose=1)
svm_search.fit(X_train_confirmed, y_train_confirmed)

In [ ]:
svm_search.best_params_

In [ ]:
years = [2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030]
future_years = np.array([years]).reshape(-1, 1)

svm_confirmed = svm_search.best_estimator_
svm_pred = svm_confirmed.predict(future_years)

In [ ]:
svm_confirmed

In [ ]:
svm_pred

In [ ]:
# check against testing data
svm_test_pred = svm_confirmed.predict(X_test_confirmed)
plt.plot(svm_test_pred)
plt.plot(y_test_confirmed)
print('MAE:', mean_absolute_error(svm_test_pred, y_test_confirmed))
print('MSE:', mean_squared_error(svm_test_pred, y_test_confirmed))

In [ ]:
# total number of cases over time

plt.figure(figsize=(20, 12))
plt.plot(future_years, svm_pred)
plt.title('Netflix Movies Added Since 2008', size=30)
plt.xlabel('Years', size=30)
plt.ylabel('NUmber of Movies Added', size=30)
plt.xticks(size=15)
plt.yticks(size=15)
plt.show()